In [16]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install opendatasets
     

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import opendatasets as od
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import gc
import re
import gzip
import time
import warnings
from dask import dataframe as dd


In [19]:
from dask import dataframe as dd
import time
st = time.time()
dask_df = dd.read_csv('/content/kill_match_stats_final_0.csv')
et = time.time()
tt = et - st
print('Dask execution time:', tt, 'seconds')

Dask execution time: 0.04918694496154785 seconds


In [23]:
from dask import dataframe as dd
import time
st = time.time()
dask_df = dd.read_csv('/content/kill_match_stats_final_0.csv')
et = time.time()
tt = et - st
print('Dask execution time:', tt, 'seconds')


Dask execution time: 0.012042045593261719 seconds


In [25]:
import pandas as pd
df = pd.read_csv('/content/kill_match_stats_final_0.csv', delimiter = ',')
df.head(5)


,killed_by,killer_name,killer_placement,killer_position_x,killer_position_y,map,match_id,time,victim_name,victim_placement,victim_position_x,victim_position_y
0,Grenade,KrazyPortuguese,5.0,657725.10,146275.2,MIRAMAR,2U4GBNA0YmnLSqvEycnTjo-KT000vfUnhSA2vfVhVPe1QB...,823.0,KrazyPortuguese,5.0,657725.10,146275.2
1,SCAR-L,nide2Bxiaojiejie,31.0,93091.37,722236.4,MIRAMAR,2U4GBNA0YmnLSqvEycnTjo-KT000vfUnhSA2vfVhVPe1QB...,194.0,X3evolution,33.0,92238.68,723375.1
2,S686,Ascholes,43.0,366921.40,421623.9,MIRAMAR,2U4GBNA0YmnLSqvEycnTjo-KT000vfUnhSA2vfVhVPe1QB...,103.0,CtrlZee,46.0,367304.50,421216.1
3,Down and Out,Weirdo7777,9.0,472014.20,313274.8,MIRAMAR,2U4GBNA0YmnLSqvEycnTjo-KT000vfUnhSA2vfVhVPe1QB...,1018.0,BlackDpre,13.0,476645.90,316758.4
4,M416,Solayuki1,9.0,473357.80,318340.5,MIRAMAR,2U4GBNA0YmnLSqvEycnTjo-KT000vfUnhSA2vfVhVPe1QB...,1018.0,Vjolt,13.0,473588.50,318418.8


In [26]:
# Remove White Spaces
df.columns = df.columns.str.replace(' ', '_')

# Remove Special Characters
df.columns = df.columns.str.replace('[?, ~, -, +, $, ^, *, #, @, &]', '')

# Inspect Columns
df.columns


<ipython-input-26-2db37872da04>:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('[?, ~, -, +, $, ^, *, #, @, &]', '')


Index(['killed_by', 'killer_name', 'killer_placement', 'killer_position_x',
       'killer_position_y', 'map', 'match_id', 'time', 'victim_name',
       'victim_placement', 'victim_position_x', 'victim_position_y'],
      dtype='object')

In [2]:
%%writefile testutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0


Writing testutility.py


In [28]:
%%writefile file.yaml
file_type: csv
dataset_name: death
file_name: kill_match_stats_final_0
table_name: death 
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - killed_by 
    - killer_name 
    - killer_placement 
    - killer_position
    - killer_position_y
    - map 
    - match_id 
    - time 
    - victim_name
    - victim_placement 
    - victim_position_x 
    - victim_position_y
    - dtype='object'
     

Writing file.yaml


In [ ]:
# Read config file
import utility as util
config_data = util.read_config_file("file.yaml")

In [34]:
import os

print(os.getcwd())

# Normal reading process of the file
import pandas as pd
#import dask.dataframe as dd

#%% time
import time
start_time = time.time()

df_sample = pd.read_csv("/content/kill_match_stats_final_0.csv",delimiter=',') 
#df_sample = pd.read_csv("/content/kill_match_stats_final_0.csv",delimiter=',') 

print("--- %s seconds ---" % (time.time() - start_time))

df_sample.head()
df_sample.info()

/content
--- 2.9665474891662598 seconds ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1609634 entries, 0 to 1609633
Data columns (total 12 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   killed_by          1609634 non-null  object 
 1   killer_name        1497577 non-null  object 
 2   killer_placement   1497577 non-null  float64
 3   killer_position_x  1497577 non-null  float64
 4   killer_position_y  1497577 non-null  float64
 5   map                1592430 non-null  object 
 6   match_id           1609634 non-null  object 
 7   time               1609634 non-null  int64  
 8   victim_name        1609634 non-null  object 
 9   victim_placement   1578914 non-null  float64
 10  victim_position_x  1609634 non-null  float64
 11  victim_position_y  1609633 non-null  float64
dtypes: float64(6), int64(1), object(5)
memory usage: 147.4+ MB


In [35]:
import dask.dataframe as dd

#%%time
import time
start_time = time.time()

#df_sample = dd.read_csv("/content/kill_match_stats_final_0.csv",delimiter=',')

df_sample = dd.read_csv("/content/kill_match_stats_final_0.csv",delimiter=',', dtype={'bed': 'object','cabin': 'object'})

    

print("--- %s seconds ---" % (time.time() - start_time))

df_sample.head()
#df_sample.info()
     

--- 0.011279582977294922 seconds ---


,killed_by,killer_name,killer_placement,killer_position_x,killer_position_y,map,match_id,time,victim_name,victim_placement,victim_position_x,victim_position_y
0,Grenade,KrazyPortuguese,5.0,657725.10,146275.2,MIRAMAR,2U4GBNA0YmnLSqvEycnTjo-KT000vfUnhSA2vfVhVPe1QB...,823,KrazyPortuguese,5.0,657725.10,146275.2
1,SCAR-L,nide2Bxiaojiejie,31.0,93091.37,722236.4,MIRAMAR,2U4GBNA0YmnLSqvEycnTjo-KT000vfUnhSA2vfVhVPe1QB...,194,X3evolution,33.0,92238.68,723375.1
2,S686,Ascholes,43.0,366921.40,421623.9,MIRAMAR,2U4GBNA0YmnLSqvEycnTjo-KT000vfUnhSA2vfVhVPe1QB...,103,CtrlZee,46.0,367304.50,421216.1
3,Down and Out,Weirdo7777,9.0,472014.20,313274.8,MIRAMAR,2U4GBNA0YmnLSqvEycnTjo-KT000vfUnhSA2vfVhVPe1QB...,1018,BlackDpre,13.0,476645.90,316758.4
4,M416,Solayuki1,9.0,473357.80,318340.5,MIRAMAR,2U4GBNA0YmnLSqvEycnTjo-KT000vfUnhSA2vfVhVPe1QB...,1018,Vjolt,13.0,473588.50,318418.8


**File Pipeline**

In [12]:
import codecs as codec
import data_sample
data_sample.to_csv(r'/content/kill_match_stats_final_0.csv', sep = '|', index=False) 

data_sample.info()
data_sample.describe()
data_sample.shape

#write dataframe df to the outputfile with pipe delimited

import gzip
import csv

f=gzip.open("/content/kill_match_stats_final_0.csv", "wt")
to_write =  open('/content/kill_match_stats_final_0.csv', 'rt') 

csv_w=csv.writer(f)



for row in to_write :
    csv_w.writerow(codec.encode(row))
f.close()
to_write.close()

ModuleNotFoundError: ignored